In [1]:
from pyairtable import Table
import pandas as pd
import json

# Airtable setup
keys_path = 'keys/'
with open(keys_path+'api_keys.json') as f:
  api_keys = json.loads(f.read())

base_id = api_keys['airtable_db']
table_name = api_keys['airtable_table']
api_token = api_keys['airtable_token']

table = Table(api_token, base_id, table_name)

# Fetch existing records from Airtable
existing_records = table.all()

/var/folders/bm/vq7gwnds0q7b4f1vgyb_jcv40000gn/T/ipykernel_51838/3028391555.py:14: DeprecationWarning: Passing API keys or base IDs to pyairtable.Table is deprecated; use Api.table() or Base.table() instead. See https://pyairtable.rtfd.org/en/latest/migrations.html for details.
  table = Table(api_token, base_id, table_name)


In [13]:
# Load the updated CSV
updated_csv_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/notebooks_ask_media/channels.csv'
df_updated = pd.read_csv(updated_csv_path, sep=';', encoding='utf-8')
df_updated.shape

(49, 10)

In [2]:
# Convert records to a DataFrame
df_channels = pd.DataFrame([record['fields'] for record in existing_records])

In [5]:
# keep only status='Active'
df_channels[df_channels['status'] == 'Active']

,media_type,website,last_id,channel_name,channel_code,link,content_type,stance,status,audience size (primary)
0,web/press,meduza.io,107806,meduzalive,MEDZ,https://t.me/meduzalive,altern,altern,Active,NaN
1,web/press,interfax.ru,45709,interfaxonline,INFX,https://t.me/interfaxonline,propaganda,inet propaganda,Active,16.0
2,tv,rt.com,205580,rt_russian,RT,https://t.me/rt_russian,propaganda,tv,Active,105.0
3,tg - pers,NaN,47456,sashakots,KOTS,https://t.me/sashakots,voenkor,voenkor,Active,NaN
4,web/press,novayagazeta.ru,45805,novaya_pishet,NGZ,https://t.me/novaya_pishet,altern,altern,Active,NaN
5,tv,NaN,34592,tvrussia1,ROS1,https://t.me/tvrussia1,propaganda,tv,Active,NaN
6,tg - news,NaN,55240,breakingmash,BRKM,https://t.me/breakingmash,moder,moder,Active,NaN
7,web/press,tass.ru,256151,tass_agency,TASS,https://t.me/tass_agency,propaganda,inet propaganda,Active,35.0
8,tg - pers,NaN,67662,voenacher,VOEN,https://t.me/voenacher,voenkor,voenkor,Active,NaN
9,web/press,tsargrad.tv,80937,tsargradtv,TSAR,https://t.me/tsargradtv,propaganda,inet propaganda,Active,34.0


In [16]:
set(df_updated.columns) - set(df_channels.columns), set(df_channels.columns) - set(df_updated.columns)

({'audience size', 'ignore'}, {'audience size (primary)'})

In [18]:
# Update last_id in Airtable
for record in existing_records:
    channel_name = record['fields'].get('channel_name')
    # Find the matching row in the updated DataFrame
    matching_row = df_updated[df_updated['channel_name'] == channel_name]
    if not matching_row.empty:
        new_last_id = matching_row['last_id'].values[0]
        # Check if last_id needs to be updated
        if str(record['fields'].get('last_id')) != str(new_last_id):
            # Update the record in Airtable
            table.update(record['id'], {'last_id': new_last_id})
            print(f"Updated last_id for channel {channel_name} to {new_last_id}")

Updated last_id for channel meduzalive to 107806.0
Updated last_id for channel interfaxonline to 45709.0
Updated last_id for channel rt_russian to 205580.0
Updated last_id for channel sashakots to 47456.0
Updated last_id for channel novaya_pishet to 45805.0
Updated last_id for channel tvrussia1 to 34592.0
Updated last_id for channel breakingmash to 55240.0
Updated last_id for channel tass_agency to 256151.0
Updated last_id for channel voenacher to 67662.0
Updated last_id for channel tsargradtv to 80937.0
Updated last_id for channel BFMnews to 43270.0
Updated last_id for channel rybar to 61164.0
Updated last_id for channel kommersant to 67937.0
Updated last_id for channel rian_ru to 250321.0
Updated last_id for channel izvestia to 176088.0
Updated last_id for channel vestiru24 to 123660.0
Updated last_id for channel rusvesnasu to 29430.0
Updated last_id for channel truekpru to 158334.0
Updated last_id for channel uranews to 95187.0
Updated last_id for channel aifonline to 102131.0
Updat

In [2]:
channel = 'uranews'
matching_records = table.all(formula=f"{{channel_name}}='{channel}'")

In [5]:
matching_records

[{'id': 'recKw2qRCv5Kgc1kq',
  'createdTime': '2024-06-16T13:52:50.000Z',
  'fields': {'media_type': 'web/press',
   'website': 'ura.news',
   'last_id': 95187,
   'channel_name': 'uranews',
   'channel_code': 'URAN',
   'link': 'https://t.me/uranews',
   'content_type': 'propaganda',
   'audience size (primary)': 50,
   'stance': 'inet propaganda'}}]

In [7]:
last_id = 95186
record_id = matching_records[0]['id']
table.update(record_id, {'last_id': last_id})

{'id': 'recKw2qRCv5Kgc1kq',
 'createdTime': '2024-06-16T13:52:50.000Z',
 'fields': {'media_type': 'web/press',
  'website': 'ura.news',
  'last_id': 95186,
  'channel_name': 'uranews',
  'channel_code': 'URAN',
  'link': 'https://t.me/uranews',
  'content_type': 'propaganda',
  'audience size (primary)': 50,
  'stance': 'inet propaganda'}}